# Using [vtreat](https://github.com/WinVector/pyvtreat) with Multinomial Classification Problems

Nina Zumel and John Mount
September 2019

Note: this is a description of the [`Python` version of `vtreat`](https://github.com/WinVector/pyvtreat), the same example for the [`R` version of `vtreat`](https://github.com/WinVector/vtreat) can be found [here](https://github.com/WinVector/pyvtreat/blob/master/Examples/Multinomial/Multinomial.md).


## Preliminaries

Load modules/packages.

In [1]:
import pkg_resources
import pandas
import numpy
import numpy.random
import seaborn
import matplotlib.pyplot as plt
import vtreat
import vtreat.util
import wvpy.util

Generate example data. 

* `y` is a noisy sinusoidal function of the variable `x`
* `yc` is the multiple class output to be predicted: : `y`'s quantized value. 
* Input `xc` is a categorical variable that represents a discretization of `y`, along some `NaN`s
* Input `x2` is a pure noise variable with no relationship to the output

In [2]:
def make_data(nrows):
    d = pandas.DataFrame({'x': 5*numpy.random.normal(size=nrows)})
    d['y'] = numpy.sin(d['x']) + 0.1*numpy.random.normal(size=nrows)
    d.loc[numpy.arange(3, 10), 'x'] = numpy.nan                           # introduce a nan level
    d['xc'] = ['level_' + str(5*numpy.round(yi/5, 1)) for yi in d['y']]
    d['x2'] = numpy.random.normal(size=nrows)
    d.loc[d['xc']=='level_-1.0', 'xc'] = numpy.nan  # introduce a nan level
    d['yc'] = numpy.where(d['y']>0.5, 'large', numpy.where(d['y']<-0.5, 'small', 'liminal'))
    return d

d = make_data(500)

d.head()

,x,y,xc,x2,yc
0,0.516951,0.503713,level_0.5,-0.756154,large
1,-5.998797,0.198779,level_0.0,0.597748,liminal
2,-2.350743,-0.713556,level_-0.5,1.497122,small
3,NaN,-0.577617,level_-0.5,-0.844595,small
4,NaN,-0.677406,level_-0.5,0.507180,small


### Some quick data exploration

Check how many levels `xc` has, and their distribution (including `NaN`)

In [3]:
d['xc'].unique()

array(['level_0.5', 'level_0.0', 'level_-0.5', nan, 'level_1.0',
       'level_-0.0', 'level_1.5'], dtype=object)

In [4]:
d['xc'].value_counts(dropna=False)

level_1.0     120
NaN           111
level_-0.5    103
level_0.5      90
level_-0.0     38
level_0.0      37
level_1.5       1
Name: xc, dtype: int64

Show the distribution of `yc`

In [5]:
d['yc'].value_counts(dropna=False)

large      170
small      170
liminal    160
Name: yc, dtype: int64

Plot conditional distribution of `x` versus `yc`.

In [6]:
# TODO: multiple histograms ( generalize wvpy dual_density_plot )

## Build a transform appropriate for classification problems.

Now that we have the data, we want to treat it prior to modeling: we want training data where all the input variables are numeric and have no missing values or `NaN`s.

First create the data treatment transform object, in this case a treatment for a binomial classification problem.

In [7]:
transform = vtreat.MultinomialOutcomeTreatment(
    outcome_name='yc',    # outcome variable
    cols_to_copy=['y'],   # columns to "carry along" but not treat as input variables
)  

Use the training data `d` to fit the transform and the return a treated training set: completely numeric, with no missing values.
Note that for the training data `d`: `transform.fit_transform()` is **not** the same as `transform.fit().transform()`; the second call can lead to nested model bias in some situations, and is **not** recommended. 
For other, later data, not seen during transform design `transform.transform(o)` is an appropriate step.

In [8]:
d_prepared = transform.fit_transform(d, d['yc'])

Now examine the score frame, which gives information about each new variable, including its type, which original variable it is  derived from, its (cross-validated) correlation with the outcome, and its (cross-validated) significance as a one-variable linear model for the outcome. 

In [9]:
transform.score_frame_

,variable,orig_variable,treatment,y_aware,has_range,PearsonR,significance,vcount,default_threshold,recommended,outcome_target
0,x_is_bad,x,missing_indicator,False,True,0.058214,1.937519e-01,2.0,0.100000,False,large
1,xc_is_bad,xc,missing_indicator,False,True,-0.383402,5.905337e-19,2.0,0.100000,True,large
2,x,x,clean_copy,False,True,0.007701,8.636164e-01,2.0,0.100000,False,large
3,x2,x2,clean_copy,False,True,-0.061960,1.665610e-01,2.0,0.100000,False,large
4,xc_logit_code_small,xc,logit_code,True,True,-0.617026,8.733543e-54,3.0,0.066667,False,large
5,xc_logit_code_liminal,xc,logit_code,True,True,-0.363087,5.010601e-17,3.0,0.066667,False,large
6,xc_logit_code_large,xc,logit_code,True,True,0.856488,3.887271e-145,3.0,0.066667,True,large
7,xc_prevalence_code,xc,prevalence_code,False,True,0.374794,4.028132e-18,1.0,0.200000,True,large
8,xc_lev_level_1.0,xc,indicator_code,False,True,0.782945,9.953930e-105,4.0,0.050000,True,large
9,xc_lev__NA_,xc,indicator_code,False,True,-0.383402,5.905337e-19,4.0,0.050000,True,large


Note that the variable `xc` has been converted to multiple variables: 

* an indicator variable for each possible level (`xc_lev_level_*`)
* the value of a (cross-validated) one-variable model for `yc` as a function of `xc` (`xc_logit_code`)
* a variable that returns how prevalent this particular value of `xc` is in the training data (`xc_prevalence_code`)
* a variable indicating when `xc` was `NaN` in the original data (`xc_is_bad`, `x_is_bad`)

Any or all of these new variables are available for downstream modeling.

The `recommended` column indicates which variables are non constant (`has_range` == True) and have a significance value smaller than `default_threshold`. See the section *Deriving the Default Thresholds* below for the reasoning behind the default thresholds. Recommended columns are intended as advice about which variables appear to be most likely to be useful in a downstream model. This advice attempts to be conservative, to reduce the possibility of mistakenly eliminating variables that may in fact be useful (although, obviously, it can still mistakenly eliminate variables that have a real but non-linear relationship to the output, as is the case with `x`, in  our example).

## Examining variables

To select variables we either make our selection in terms of new variables as follows.

In [10]:
score_frame = transform.score_frame_
good_new_variables = score_frame.variable[score_frame.recommended].unique()
good_new_variables

array(['xc_is_bad', 'xc_logit_code_large', 'xc_prevalence_code',
       'xc_lev_level_1.0', 'xc_lev__NA_', 'xc_lev_level_-0.5',
       'xc_lev_level_0.5', 'x_is_bad', 'xc_logit_code_liminal',
       'xc_logit_code_small'], dtype=object)

Or in terms of original variables as follows.

In [11]:
good_original_variables = score_frame.orig_variable[score_frame.recommended].unique()
good_original_variables

array(['xc', 'x'], dtype=object)

Notice, in each case we must call unique as each variable (derived or original) is potentially qualified against each possible outcome.

Notice that, by default, `d_prepared` only includes recommended variables (along with `y` and `yc`):

In [12]:
d_prepared.head()

,y,yc,x_is_bad,xc_is_bad,xc_logit_code_small,xc_logit_code_liminal,xc_logit_code_large,xc_prevalence_code,xc_lev_level_1.0,xc_lev__NA_,xc_lev_level_-0.5,xc_lev_level_0.5
0,0.503713,large,0.0,0.0,-5.798324,0.373600,0.433650,0.180,0.0,0.0,0.0,1.0
1,0.198779,liminal,0.0,0.0,-5.732316,1.084128,-5.767509,0.074,0.0,0.0,0.0,0.0
2,-0.713556,small,0.0,0.0,0.535932,0.243899,-5.804795,0.206,0.0,0.0,1.0,0.0
3,-0.577617,small,1.0,0.0,0.455023,0.349195,-5.815070,0.206,0.0,0.0,1.0,0.0
4,-0.677406,small,1.0,0.0,0.459433,0.346323,-5.806387,0.206,0.0,0.0,1.0,0.0


This is `vtreat`s default behavior; to include all variables in the prepared data, set the parameter `filter_to_recommended` to False, as we show later, in the *Parameters for `BinomialOutcomeTreatment`* section below.


## A Closer Look at `logit_code` variables

Variables of type `logit_code` are the outputs of a one-variable hierarchical logistic regression of a categorical variable (in our example, `xc`) against the centered output on the (cross-validated) treated training data. 

Let's see whether `xc_logit_code` makes a good one-variable model for `yc`. It has a large AUC:

In [13]:
# TODO: figure out a presentation
# wvpy.util.plot_roc(prediction=d_prepared['xc_logit_code'], 
#                   istrue=d_prepared['yc'],
#                   title = 'performance of xc_logit_code variable')

This indicates that `xc_logit_code` is strongly predictive of the outcome. Negative values of `xc_logit_code` correspond strongly to negative outcomes, and positive values correspond strongly to positive outcomes.

In [14]:
# TODO: figure out a presentation
# wvpy.util.dual_density_plot(probs=d_prepared['xc_logit_code'], 
#                            istrue=d_prepared['yc'])

The values of `xc_logit_code_*` are in "link space". We can often visualize the relationship a little better by converting the logistic score to a probability.

In [15]:
from scipy.special import expit  # sigmoid
from scipy.special import logit

# TODO: figure out presenetation
# offset = logit(numpy.mean(d_prepared.yc))
# wvpy.util.dual_density_plot(probs=expit(d_prepared['xc_logit_code_large'] + offset),
#                            istrue=d_prepared['yc'])                                   

Variables of type `logit_code` are useful when dealing with categorical variables with a very large number of possible levels. For example, a categorical variable with 10,000 possible values potentially converts to 10,000 indicator variables, which may be unwieldy for some modeling methods. Using a single numerical variable of type `logit_code` may be a preferable alternative.

## Using the Prepared Data in a Model

Of course, what we really want to do with the prepared training data is to fit a model jointly with all the (recommended) variables. 
Let's try fitting a logistic regression model to `d_prepared`.

In [16]:
import sklearn.linear_model
import seaborn

not_variables = ['y', 'yc', 'prediction']
model_vars = [v for v in d_prepared.columns if v not in set(not_variables)]

fitter = sklearn.linear_model.LogisticRegression(
    solver = 'saga',
    multi_class = 'multinomial')
fitter.fit(d_prepared[model_vars], d_prepared['yc'])


# now predict

def add_predictions(d_prepared, model_vars, fitter):
    pred = fitter.predict_proba(d_prepared[model_vars])
    classes = fitter.classes_
    for i in range(len(classes)):
        d_prepared[classes[i]] = pred[:, i]
    return d_prepared


d_prepared = add_predictions(d_prepared, model_vars, fitter)
d_prepared.head()

/Users/johnmount/anaconda3/envs/aiAcademy/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


,y,yc,x_is_bad,xc_is_bad,xc_logit_code_small,xc_logit_code_liminal,xc_logit_code_large,xc_prevalence_code,xc_lev_level_1.0,xc_lev__NA_,xc_lev_level_-0.5,xc_lev_level_0.5,large,liminal,small
0,0.503713,large,0.0,0.0,-5.798324,0.373600,0.433650,0.180,0.0,0.0,0.0,1.0,0.521490,0.477876,0.000634
1,0.198779,liminal,0.0,0.0,-5.732316,1.084128,-5.767509,0.074,0.0,0.0,0.0,0.0,0.000231,0.998479,0.001290
2,-0.713556,small,0.0,0.0,0.535932,0.243899,-5.804795,0.206,0.0,0.0,1.0,0.0,0.000645,0.419993,0.579362
3,-0.577617,small,1.0,0.0,0.455023,0.349195,-5.815070,0.206,0.0,0.0,1.0,0.0,0.000943,0.203087,0.795970
4,-0.677406,small,1.0,0.0,0.459433,0.346323,-5.806387,0.206,0.0,0.0,1.0,0.0,0.000952,0.202038,0.797009


In [17]:
# TODO: work out a presentation
# look at the ROC curve (on the training data)
#wvpy.util.plot_roc(prediction=d_prepared['prediction'], 
#                   istrue=d_prepared['yc'],
#                   title = 'Performance of logistic regression model on training data')

Now apply the model to new data.

In [18]:
# create the new data
dtest = make_data(450)

# prepare the new data with vtreat
dtest_prepared = transform.transform(dtest)

# apply the model to the prepared data
dtest_prepared = add_predictions(dtest_prepared, model_vars, fitter)

In [19]:
wvpy.util.plot_roc(prediction=dtest_prepared['prediction'], 
                   istrue=dtest_prepared['yc'],
                   title = 'Performance of logistic regression model on test data')

KeyError: 'prediction'

## Parameters for `BinomialOutcomeTreatment`

We've tried to set the defaults for all parameters so that `vtreat` is usable out of the box for most applications.


In [ ]:
vtreat.vtreat_parameters()

**use_hierarchical_estimate:**: When True, uses hierarchical smoothing when estimating `logit_code` variables; when False, uses unsmoothed logistic regression.

**coders**: The types of synthetic variables that `vtreat` will (potentially) produce. See *Types of prepared variables* below.

**filter_to_recommended**: When True, prepared data only includes variables marked as "recommended" in score frame. When False, prepared data includes all variables. See the Example below.

**indicator_min_fraction**: For categorical variables, indicator variables (type `indicator_code`) are only produced for levels that are present at least `indicator_min_fraction` of the time. A consequence of this is that 1/`indicator_min_fraction` is the maximum number of indicators that will be produced for a given categorical variable. To make sure that *all* possible indicator variables are produced, set `indicator_min_fraction = 0`

**cross_validation_plan**: The cross validation method used by `vtreat`. Most people won't have to change this.

**cross_validation_k**: The number of folds to use for cross-validation

**user_transforms**: For passing in user-defined transforms for custom data preparation. Won't be needed in most situations, but see [here](https://github.com/WinVector/pyvtreat/blob/master/Examples/UserCoders/UserCoders.ipynb) for an example of applying a GAM transform to input variables.

**sparse_indicators**: When True, use a (Pandas) sparse representation for indicator variables. This representation is compatible with `sklearn`; however, it may not be compatible with other modeling packages. When False, use a dense representation.

### Example: Use all variables to model, not just recommended

In [ ]:
transform_all = vtreat.BinomialOutcomeTreatment(
    outcome_name='yc',    # outcome variable
    outcome_target=True,  # outcome of interest
    cols_to_copy=['y'],   # columns to "carry along" but not treat as input variables
    params = vtreat.vtreat_parameters({
        'filter_to_recommended': False
    })
)  

transform_all.fit_transform(d, d['yc']).columns

In [ ]:
transform_all.score_frame_

Note that the prepared data produced by `fit_transform()` includes all the variables, including those that were not marked as "recommended". 

## Types of prepared variables

**clean_copy**: Produced from numerical variables: a clean numerical variable with no `NaNs` or missing values

**indicator_code**: Produced from categorical variables, one for each (common) level: for each level of the variable, indicates if that level was "on"

**prevalence_code**: Produced from categorical variables: indicates how often each level of the variable was "on"

**logit_code**: Produced from categorical variables: score from a one-dimensional model of the centered output as a function of the variable

**missing_indicator**: Produced for both numerical and categorical variables: an indicator variable that marks when the original variable was missing or  `NaN`

**deviation_code**: not used by `BinomialOutcomeTreatment`

**impact_code**: not used by `BinomialOutcomeTreatment`

### Example: Produce only a subset of variable types

In this example, suppose you only want to use indicators and continuous variables in your model; 
in other words, you only want to use variables of types (`clean_copy`, `missing_indicator`, and `indicator_code`), and no `logit_code` or `prevalence_code` variables.

In [ ]:
transform_thin = vtreat.BinomialOutcomeTreatment(
    outcome_name='yc',    # outcome variable
    outcome_target=True,  # outcome of interest
    cols_to_copy=['y'],   # columns to "carry along" but not treat as input variables
    params = vtreat.vtreat_parameters({
        'filter_to_recommended': False,
        'coders': {'clean_copy',
                   'missing_indicator',
                   'indicator_code',
                  }
    })
)

transform_thin.fit_transform(d, d['yc']).head()

In [ ]:
transform_thin.score_frame_

## Deriving the Default Thresholds

While machine learning algorithms are generally tolerant to a reasonable number of irrelevant or noise variables, too many irrelevant variables can lead to serious overfit; see [this article](http://www.win-vector.com/blog/2014/02/bad-bayes-an-example-of-why-you-need-hold-out-testing/) for an extreme example, one we call "Bad Bayes". The default threshold is an attempt to eliminate obviously irrelevant variables early.

Imagine that you have a pure noise dataset, where none of the *n* inputs are related to the output. If you treat each variable as a one-variable model for the output, and look at the significances of each model, these significance-values will be uniformly distributed in the range [0:1]. You want to pick a weakest possible significance threshold that eliminates as many noise variables as possible. A moment's thought should convince you that a threshold of *1/n* allows only one variable through, in expectation. 

This leads to the general-case heuristic that a significance threshold of *1/n* on your variables should allow only one irrelevant variable through, in expectation (along with all the relevant variables). Hence, *1/n* used to be our recommended threshold, when we developed the R version of `vtreat`.

We noticed, however, that this biases the filtering against numerical variables, since there are at most two derived variables (of types *clean_copy* and *missing_indicator* for every numerical variable in the original data. Categorical variables, on the other hand, are expanded to many derived variables: several indicators (one for every common level), plus a *logit_code* and a *prevalence_code*. So we now reweight the thresholds. 

Suppose you have a (treated) data set with *ntreat* different types of `vtreat` variables (`clean_copy`, `indicator_code`, etc).
There are *nT* variables of type *T*. Then the default threshold for all the variables of type *T* is *1/(ntreat nT)*. This reweighting  helps to reduce the bias against any particular type of variable. The heuristic is still that the set of recommended variables will allow at most one noise variable into the set of candidate variables.

As noted above, because `vtreat` estimates variable significances using linear methods by default, some variables with a non-linear relationship  to the output may fail to pass the threshold. Setting the `filter_to_recommended` parameter to False will keep all derived variables in the treated frame, for the data scientist to filter (or not) as they will.



## Conclusion

In all cases (classification, regression, unsupervised, and multinomial classification) the intent is that `vtreat` transforms are essentially one liners.

The preparation commands are organized as follows:

 * **Regression**: [`R` regression example](https://github.com/WinVector/vtreat/blob/master/Examples/Regression/Regression.md), [`Python` regression example](https://github.com/WinVector/pyvtreat/blob/master/Examples/Regression/Regression.md).
 * **Classification**: [`R` classification example](https://github.com/WinVector/vtreat/blob/master/Examples/Classification/Classification.md), [`Python` classification  example](https://github.com/WinVector/pyvtreat/blob/master/Examples/Multinomial/Multinomial.md).
 * **Unsupervised tasks**: [`R` unsupervised example](https://github.com/WinVector/vtreat/blob/master/Examples/Unsupervised/Unsupervised.md), [`Python` unsupervised example](https://github.com/WinVector/pyvtreat/blob/master/Examples/Unsupervised/Unsupervised.md).
 * **Multinomial classification**: [`R` multinomial classification example](https://winvector.github.io/vtreat/articles/MultiClassVtreat.html), [`Python` multinomial classification example](https://github.com/WinVector/pyvtreat/blob/master/Examples/Multinomial/MultinomialExample.ipynb).

These current revisions of the examples are designed to be small, yet complete.  So as a set they have some overlap, but the user can rely mostly on a single example for a single task type.
